In [37]:
%matplotlib qt                           
import numpy as np                    
import matplotlib as mp
import math
import matplotlib.pyplot as plt
import scipy
from scipy.stats import truncnorm

#Define detector and create grid completely using functions 
class detector():
    def __init__(self, centre, side_len, rot, pix_len):    #Assuming square pixels, rot = Angle through which detector is rotated along z = z_centre
        self.centre = centre              #Array (1,3),     centre: Array(3)
        self.side_x = side_len[0]         #Array Element,   side_len: Array(2)
        self.side_y = side_len[1]         #Array Element    
        self.rot    = math.radians(rot)        #Lattitudinal Angle
        self.pix_len_x= pix_len[0]        #Array Element,   pix_len:Array(2)
        self.pix_len_y= pix_len[1]        #Array Element  
        self.pix_id = np.arange((side_len[0]/pix_len[0])*(side_len[1]/pix_len[1]))    #Pixel Index
        self.pix_area = pix_len[0]*pix_len[1]
        #rename 'self.pix_norm' to 'self.pix_area_vect'
        self.pix_norm = np.array([math.sin(self.rot), 0 , -math.cos(self.rot)])*(self.pix_area)   #Directed below the detector (negative z direction) 
        
        #X coordinates of pixels
        self.pix_x = np.linspace((self.centre[0] - (self.side_x/2)*math.cos(self.rot)), (self.centre[0] + (self.side_x/2)*math.cos(self.rot)), int(self.side_x/self.pix_len_x))
        
        #Y coordinates of pixels
        self.pix_y = np.linspace((self.centre[1] - (self.side_y/2)), (self.centre[1] + (self.side_y/2)), int(self.side_y/self.pix_len_y))
        
        #z coordinates of pixels
        self.pix_z = np.linspace((self.centre[2] - (self.side_x/2)*math.sin(self.rot)), (self.centre[2] + (self.side_x/2)*math.sin(self.rot)), int(self.side_x/self.pix_len_x))
        
        
        self.pix_x_mesh, self.pix_y_mesh = np.meshgrid(self.pix_x, self.pix_y)     #Mesh Coordinates for 3D Plotting
        self.pix_z_mesh, self.pix_y_mesh = np.meshgrid(self.pix_z, self.pix_y)     #Mesh Coordinates for 3D Plotting
        
        self.pix_x_mesh_flat = (self.pix_x_mesh).reshape(len(self.pix_x)*len(self.pix_y))  #Flattened Mesh x
        self.pix_y_mesh_flat = (self.pix_y_mesh).reshape(len(self.pix_x)*len(self.pix_y))  #Flattened Mesh y
        self.pix_z_mesh_flat = (self.pix_z_mesh).reshape(len(self.pix_z)*len(self.pix_y))  #Flattened Mesh z

    
    def plot_pix(self):   #Graph for displaying pixels
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(self.pix_x_mesh, self.pix_y_mesh, self.pix_z_mesh)
        plt.show()
        
    def conv_1(self,t):   #Conversion of flat mesh array parameter into coordinates of pixels wrt corner of detector
        return tuple(reversed(divmod(t, len(self.pix_x))))
    
    def conv_2(self,x,y):  #Conversion of x,y(wrt to corner of detector) into flat mesh array parameter 
        return (y*len(self.pix_x) + x)   


CZT = detector([5, 0, 10], [16, 20], 40, [1, 1])
NaI = detector([0, 0, 0], [20, 20], 0, [1, 1])
incident_photon_dir = CZT.pix_norm
#incident_photon_dir = [0,0,-1]   #Direction(Unit vector) of the incident beam of photons
counts_per_pix= 1e7  #Number of photons incident on each CZT pixel 

#------------------------------------------------------------------------------------------------------------------------------
#Plot both detector pixels in one graph    
def plot_both_det(): 
    
    '''This function plots both the detectors simultaneously'''
    
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    czt = ax.scatter(CZT.pix_x_mesh, CZT.pix_y_mesh, CZT.pix_z_mesh) 
    nai = ax.scatter(NaI.pix_x_mesh, NaI.pix_y_mesh, NaI.pix_z_mesh) 
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.legend((czt, nai), ('CZT', 'NaI'))
    plt.show()
    
#-----------------------------------------------------------------------------------------------------------------------------

def sol_ang_cart(x, y, z):      #Array containing all solid angles
    
    '''This function returns values of solid angles subtended by all NaI pixels at a point (x, y, z), in the form of a
       2 dimensional numpy array and a 1 dimensional flat array '''
    
    vect_mesh = np.array([NaI.pix_x_mesh- x, NaI.pix_y_mesh- y, NaI.pix_z_mesh- z])
    vect_mesh_flat = vect_mesh.reshape(3,1,len(NaI.pix_x)*len(NaI.pix_y))  
    proj_area= np.zeros(len(NaI.pix_x)*len(NaI.pix_y))
    solid_ang_flat = np.zeros(len(NaI.pix_x)*len(NaI.pix_y))
    for j in range(len(NaI.pix_x)*len(NaI.pix_y)):
        vect_f = np.array([vect_mesh_flat[0][0][j], vect_mesh_flat[1][0][j], vect_mesh_flat[2][0][j]])
        proj_area[j] = np.dot(NaI.pix_norm, (vect_f/np.linalg.norm(vect_f)))
        solid_ang_flat[j]= proj_area[j]/np.square(np.linalg.norm(vect_f))
        solid_ang = solid_ang_flat.reshape(len(NaI.pix_y), len(NaI.pix_x))
    return solid_ang, solid_ang_flat    

def sol_ang_prob_cart(x, y, z):   #If photon is scattered from a point (x,y,z),not constrained to lie on the CZT surface, then where is it most likely to hit the NaI detector 
   
    '''This function returns the probability of a photon being detected at any NaI pixel when scattered from the point (x,y,z)
       in the form of a 2 dimensional numpy array and a 1 dimensional flat numpy array. It also returns the maximum of the probabilities'''
    '''The pixel corresponding to the maxima is the pixel where the photon is most likely to hit the NaI detector'''
    
    solid_angle_prob= (sol_ang_cart(x, y, z)[0])/(4*math.pi)    
    maximum_prob = max((sol_ang_cart(x, y, z)[1])/(4*math.pi))    
    solid_angle_prob_flat = (sol_ang_cart(x, y, z)[1])/(4*math.pi)    
    return solid_angle_prob, solid_angle_prob_flat,  maximum_prob

def plot_sol_ang_prob_cart(x, y, z):           #Function for plotting solid angle probability
    
    '''This function plots the above calculated solid angle probability, or the probability due to projection, for all pixels in the NaI detector'''
    
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(NaI.pix_x_mesh ,NaI.pix_y_mesh, sol_ang_prob_cart(x, y, z)[0], rstride= 1, cstride= 1)
    ax.set_xlabel('Pixel X')
    ax.set_ylabel('Pixel Y')
    ax.set_zlabel('Solid Angle Probability')
    plt.show()    
    
def sol_ang_CZT_i(i):
    return sol_ang_cart(CZT.pix_x_mesh_flat[i], CZT.pix_y_mesh_flat[i], CZT.pix_z_mesh_flat[i])

def prob_1_CZT_i(i):             #Prob distribution with flat array parameteras input 
    
    '''Convention followed throughout- 
       -There are 2 probabilities of interest. Solid angle probability and Compton probability. The numbers 1 and 2 are used to identify which
        probability the function is refering to.
       -The input coordinate/array parameter could be that of a CZT pixel or NaI pixel. This is mentioned in the function name. 
       -The flat array index 'i' is used for CZT arrays, and 'j' for NaI arrays.
       -Functions with flat array parameter/ index of array (ie. 'i' or 'j')as input will be mentioned in the function name itself.
       -Same done for 2 dimensional input. Example: Cartesian input mentioned in name as 'cart'
       '''
    
    return sol_ang_prob_cart(CZT.pix_x_mesh_flat[i], CZT.pix_y_mesh_flat[i], CZT.pix_z_mesh_flat[i])

def prob_1_CZT_cart(x,y):     #Prob distribution with cartesian coordinates as input 
    """
    Prob distribution with relative cartesian coordinates as input.
    The coordinates input should be relative to the detector. The lower left vertex of the detector should be considered as origin. 
    """
    i = CZT.conv_2(x,y)  
    return prob_1_CZT_i(i)

def plot_prob_1_CZT_cart(x,y):
    '''Plotting the above probability'''
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(NaI.pix_x_mesh ,NaI.pix_y_mesh, prob_1_CZT_cart(x,y)[0], rstride= 1, cstride= 1)
    ax.set_xlabel('Pixel X')
    ax.set_ylabel('Pixel Y')
    ax.set_zlabel('Probability')
    plt.show()    

#--------------------------------------------------------------------------------------------------------------------------

#First detector calculations
def theta_sc():             
    '''Function for calculating scatterred angle(in degrees) for all pairs of CZT and NaI pixels; Gives entire scattered angle data in a 2D array'''
    theta_scattered = np.zeros((len(CZT.pix_x_mesh_flat), len(NaI.pix_x_mesh_flat)))
    for i in range(len(CZT.pix_x_mesh_flat)):
        for j in range(len(NaI.pix_x_mesh_flat)):
            theta_scattered[i][j] = np.degrees(math.acos(np.dot([NaI.pix_x_mesh_flat[j] - CZT.pix_x_mesh_flat[i], NaI.pix_y_mesh_flat[j]- CZT.pix_y_mesh_flat[i], NaI.pix_z_mesh_flat[j]- CZT.pix_z_mesh_flat[i]], incident_photon_dir)/np.linalg.norm([NaI.pix_x_mesh_flat[j] - CZT.pix_x_mesh_flat[i], NaI.pix_y_mesh_flat[j]- CZT.pix_y_mesh_flat[i], NaI.pix_z_mesh_flat[j]- CZT.pix_z_mesh_flat[i]])))
    return theta_scattered

def theta_sc_CZT_i(i):           
    '''Function for calculating scatterred angle for a photon scatterd from a particular postion on CZT.
       Output in the form of a 2 dimensional numpy array, and 1 dimensional flat array. The function also returns range of scattering angle'''
    theta_scattered_i_flat = np.zeros(len(NaI.pix_x_mesh_flat))
    for j in range(len(NaI.pix_x_mesh_flat)):
            theta_scattered_i_flat[j] = np.degrees(math.acos(np.dot([NaI.pix_x_mesh_flat[j] - CZT.pix_x_mesh_flat[i], NaI.pix_y_mesh_flat[j]- CZT.pix_y_mesh_flat[i], NaI.pix_z_mesh_flat[j]- CZT.pix_z_mesh_flat[i]], incident_photon_dir)/np.linalg.norm([NaI.pix_x_mesh_flat[j] - CZT.pix_x_mesh_flat[i], NaI.pix_y_mesh_flat[j]- CZT.pix_y_mesh_flat[i], NaI.pix_z_mesh_flat[j]- CZT.pix_z_mesh_flat[i]])))
    theta_scattered = theta_scattered_i_flat.reshape(len(NaI.pix_y), len(NaI.pix_x))
    theta_range = max(theta_scattered_i_flat) - min(theta_scattered_i_flat)
    return theta_scattered, theta_scattered_i_flat, theta_range

def theta_sc_CZT_cart(x, y):    #Same function as theta_sc_CZT_i with cartesian inputs          
    '''Function for calculating scattering angle with relative cartesian inputs'''
    i = CZT.conv_2(x,y)  
    return theta_sc_CZT_i(i)

def plot_theta_sc_CZT_cart(x,y):     #For plotting scattered angle      
    '''Function plots the scattered angle for a particular CZT pixel as the scattering centre'''
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface( NaI.pix_x_mesh, NaI.pix_y_mesh, (theta_sc_CZT_cart(x,y)[0]) , rstride= 3, cstride= 3)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Scattering Angle (Degrees)')
    plt.show()

#-----------------------------------------------------------------------------------------------------------------------------

#Energy scattered 
energy_i_val = np.arange(20, 201, 1)            #Range of energy of incident photon in keV
constant_1= 511                    #constant_1(Rest energy of electron in keV) used to calculate scattered angle 


#If a photon of a particular energy(e) hits a particular CZT pixel(i) then what is energy distribution after collision.
def energy_sc_CZT_i(e, i):   #Energy of Scattered Photon/ Received energy
    '''Function for calculating the energy of all scattered photons, with incident energy 'e' and 'i'th CZT pixel as the scattering centre.
       Output in the form of 2 dimensional numpy array as well as a 1 dimensional flat array'''
    e_rep = np.repeat(e, len(NaI.pix_x)*len(NaI.pix_y))
    energy_scattered_flat = e_rep/(1+ (e_rep/constant_1)*(1-np.cos(np.radians((theta_sc_CZT_i(i))[1]))))
    energy_scattered = energy_scattered_flat.reshape(len(NaI.pix_y), len(NaI.pix_x))
    return energy_scattered, energy_scattered_flat

def energy_sc_CZT_cart(e, x, y):
    '''The above energy with relative cartesian coordinates as input'''
    i = CZT.conv_2(x,y) 
    return energy_sc_CZT_i(e, i)

def plot_energy_sc_CZT_cart(e, x, y):
    '''Function for plotting the enrgy of scattered photon with relative cartesian coordinates as input'''
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(NaI.pix_x_mesh, NaI.pix_y_mesh, energy_sc_CZT_cart(e , x, y)[0])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Energy of Scattered Photon (keV)')
    plt.show()
    
def delta_energy_CZT_i(e, i):
    '''Function for calculating difference in energies of photons before and after scattering'''
    energy_diff_flat = np.repeat(e, len(NaI.pix_x)*len(NaI.pix_y)) - energy_sc_CZT_i(e, i)[1]
    energy_diff= energy_diff_flat.reshape(len(NaI.pix_y), len(NaI.pix_x))
    return energy_diff, energy_diff_flat 

def delta_energy_CZT_cart(e, x, y):
    '''Function for difference in energy with relative cartesian coordinates as input'''
    i = CZT.conv_2(x,y) 
    return delta_energy_CZT_i(e, i)

def plot_delta_energy_CZT_cart(e, x, y):
    ''''''
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(NaI.pix_x_mesh, NaI.pix_y_mesh, delta_energy_CZT_cart(e, x, y)[0])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Energy Difference (keV)')
    plt.show()

#------------------------------------------------------------------------------------------------------------------------------
'''This section contains unsused functions'''
'''These functions were used to generate random gaussian distribution of incident energies with a specific mean, lower limit,
   upper limit, and a standard deviation. 
   It is assumed in this code that the incident photons are monochromatic (ie. equal energy) hence the following functions are
   not required'''
#Energy incident distribution would be an array containing values ranging from 20 to 200 keV. It could be a flat array or an array with the same shape as CZT.pix_x_mesh/CZT.pix_y_mesh
# Gaussian/Normal Distribution of Incident Energy

def get_truncated_normal(mean, sd, low, upp):    #Truncated gaussian/normal
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
     
def energy_i_guass_dist():                  #Flat array as output
    X = get_truncated_normal(mean=130, sd=27, low=20, upp=200)
    enrgy_incident= X.rvs(len(CZT.pix_x)*len(CZT.pix_y))
    return enrgy_incident

def plot_hist():                      #Plot histogram
    plt.hist(energy_i_guass_dist(), bins=181)

#------------------------------------------------------------------------------------------------------------------------------
#Klein- Nishina Formula
#For now, assume that there is just one electron per pixel from which the photons scatter

constant_2 = 7.9406e-30   #Refer Klein-Nishina Formula 
electron_density = 1e27   #Currently set to an arbritary value
def delta_sigma(e,i,j):
    energy_ratio = (energy_sc_CZT_i(e, i)[1][j])/e
    delta_sigma = (sol_ang_CZT_i(i)[1][j])*constant_2*(energy_ratio**2)*(energy_ratio + energy_ratio**(-1) - (math.sin(np.radians(theta_sc_CZT_i(i)[1][j])))**2)
    return delta_sigma

def prob_2_CZT_NaI_i_j(e,i,j):
    return delta_sigma(e,i,j)*electron_density

def prob_2_CZT_i(e, i):
    prob_2_CZT_i_arr_flat = np.zeros(len(NaI.pix_x)*len(NaI.pix_y))
    for j in range(len(NaI.pix_x)*len(NaI.pix_y)):
        prob_2_CZT_i_arr_flat[j]= prob_2_CZT_NaI_i_j(e, i, j)
    prob_arr_2_i_arr_2d = prob_2_CZT_i_arr_flat.reshape(len(NaI.pix_y), len(NaI.pix_x))
    return prob_arr_2_i_arr_2d, prob_2_CZT_i_arr_flat

def prob_2_CZT_cart(e, x, y):
    i = CZT.conv_2(x,y)
    return prob_2_CZT_i(e, i)

def plot_prob_2_CZT_cart(e, x, y):
    i = CZT.conv_2(x,y)
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(NaI.pix_x_mesh, NaI.pix_y_mesh, prob_2_CZT_i(e, i)[0])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Probability 2')
    plt.show()    
#-------------------------------------------------------------------------------------------------------------------------------  
#Final Distribution of photons on the NaI detector 

def combined_prob_CZT_cart(e, x, y):
    '''This function returns the final probability'''
    return prob_2_CZT_cart(e, x, y)[0]*prob_1_CZT_cart(x,y)[0], prob_2_CZT_cart(e, x, y)[1]*prob_1_CZT_cart(x,y)[1]

def plot_combined_prob_CZT_cart(e,x,y):   
    '''The function below plots the probability that a photon, which is incident on the specified CZT pixel with energy 'e', would be received on each of the NaI detectors. 
    In other words, the function plots the fraction of photons received by the NaI pixels below. 
    '''
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface( NaI.pix_x_mesh, NaI.pix_y_mesh, prob_2_CZT_cart(e, x, y)[0]*prob_1_CZT_cart(x,y)[0] , rstride= 3, cstride= 3)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Probaility')
    plt.show()

def plot_combined_prob_heat_map_CZT_cart(e, x, y):
    '''
    This function plots the corresponding heat map for the combined probabilities for better representation.
    This is similiar to the 3D plot of the probabilities given by the function.
    '''
    plt.figure()
    plt.imshow(combined_prob_CZT_cart(e, x, y)[0], origin = 'lower')
    plt.show()
#------------------------------------------------------------------------------------------------------------------------------   
#Spectrum Calculations
def theta_sc_NaI_j(j):       
    '''This function calculates scattering angles for all photons which scattered from all CZT pixels and are detected by NaI pixel with index 'j'.
       The function returns a 1 dimensional array with all angles.'''
    theta_scattered = np.zeros((len(CZT.pix_x_mesh_flat)))
    for i in range(len(CZT.pix_x_mesh_flat)):
        theta_scattered[i] = np.degrees(math.acos(np.dot([NaI.pix_x_mesh_flat[j] - CZT.pix_x_mesh_flat[i], NaI.pix_y_mesh_flat[j]- CZT.pix_y_mesh_flat[i], NaI.pix_z_mesh_flat[j]- CZT.pix_z_mesh_flat[i]], incident_photon_dir)/np.linalg.norm([NaI.pix_x_mesh_flat[j] - CZT.pix_x_mesh_flat[i], NaI.pix_y_mesh_flat[j]- CZT.pix_y_mesh_flat[i], NaI.pix_z_mesh_flat[j]- CZT.pix_z_mesh_flat[i]])))
    return theta_scattered

def energy_sc_NaI_j(e, j):
    '''This function calculates the energies of photons with incident energy 'e',  after scattering from all CZT pixels, received by NaI pixel with index 'j'.
       The function returns a single 1 dimensional array with all energy values.''' 
    e_rep = np.repeat(e, len(CZT.pix_x)*len(CZT.pix_y))
    energy_scattered_j = e_rep/(1+ (e_rep/constant_1)*(1-np.cos(np.radians(theta_sc_NaI_j(j)))))
    return energy_scattered_j

def num_phot_NaI_j(e, j):
    '''This function returns the number of photons received by a NaI pixel with index 'j' after scattering from all CZT pixels. 
       The function returns a 1 dimensional array with the number of photons received from each CZT pixel.'''
    num_phot = np.zeros((len(CZT.pix_x_mesh_flat)))
    for i in range(len(CZT.pix_x_mesh_flat)):
        vector = (NaI.pix_x_mesh_flat[j]-CZT.pix_x_mesh_flat[i] , NaI.pix_y_mesh_flat[j]-CZT.pix_y_mesh_flat[i], NaI.pix_z_mesh_flat[j]-CZT.pix_z_mesh_flat[i])
        sol_ang_j = np.dot(NaI.pix_norm, vector)/(np.power(np.linalg.norm(vector), 3))
        prob_j = sol_ang_j/(4*math.pi)
        num_phot[i] = counts_per_pix*(prob_j)*prob_2_CZT_NaI_i_j(e,i, j)
    return num_phot

def sorted_data(e, j):
    '''This function is not used in the code but helps interpret data in a easier fashion.
    The function sorts the energies of photons and number of photons received by the specified NaI pixel in the for of a 
    2 dimensional array arranged in ascending order of energies'''
    double_data = np.zeros((len(CZT.pix_x_mesh_flat), 2))
    for i in range(len(CZT.pix_x_mesh_flat)):
        vector = (NaI.pix_x_mesh_flat[j]-CZT.pix_x_mesh_flat[i] , NaI.pix_y_mesh_flat[j]-CZT.pix_y_mesh_flat[i], NaI.pix_z_mesh_flat[j]-CZT.pix_z_mesh_flat[i])
        sol_ang_j = np.dot(NaI.pix_norm, vector)/(np.power(np.linalg.norm(vector), 3))
        prob_j = sol_ang_j/(4*math.pi)
        num_phot = counts_per_pix*(prob_j)*prob_2_CZT_NaI_i_j(e,i, j)
        double_data[i] = [energy_sc_NaI_j(e, j)[i], num_phot]
    return sorted(double_data, key=lambda x : x[0]) 

def plot_spec_NaI_cart(e, x, y):
    '''This function plots the spectrum for incident  energy 'e' , ie. Number of photons of same energy received, vs, the energy of those photons with same energy, received by a NaI pixel 'j'  
    The function returns a scatter plot.
    '''
    j = NaI.conv_2(x, y)
    plt.scatter(energy_sc_NaI_j(e,j), num_phot_NaI_j(e, j))
    plt.title('Spectrum ')
    plt.xlabel('Energy of Scattered Photon (keV)')
    plt.ylabel('Number of photons')
    plt.show()
    
    
def plot_hist_NaI_cart(e, x, y):
    '''
    This function puts the spectrum data in the form of a histogram with 50 bins. 
    The number of bins can be changed to the desired value.
    The histogram takes the number of photons as weight for each energy recevied. 
    '''
    j = NaI.conv_2(x, y)
    plt.figure()
    plt.hist(energy_sc_NaI_j(e, j), bins = 50, weights = num_phot_NaI_j(e, j), histtype = 'bar')
    plt.xlabel('Energy (keV)')
    plt.ylabel('Number of Photons')
    plt.show()

def mult_spec_NaI_cart_energy(energies, x,y):
    '''This function plots spectrums for 6 multiple energies together for a NaI pixel with relative cartesian coordinates (x,y).
       These energies are to included in a list/1D array.
       '''
    for k in range(6):
        plot_spec_NaI_cart(energies[k],x, y)     
    plt.legend([ energies[0] ,  energies[1] , energies[2] , energies[3] , energies[4], energies[5] ])

def mult_spec_NaI_cart_position(positions, e):
    '''This function returns scatter plots for various positions corersponding to a specific incident energy 'e'.
    These positions need to be mentioned in the form a two dimensional array
    '''
    for k in range(6):
        plt.subplot(2,3,k+1)
        plot_spec_NaI_cart(e ,positions[k][0], positions[k][1])     
        plt.legend([ (positions[k][0], positions[k][1])])       


In [38]:
plot_both_det()

In [19]:
mult_spec_NaI_cart_energy([20,80,120,150,180,200], 10,10)


In [3]:
mult_spec_NaI_cart_position([[0,0], [0,10], [10,0], [10,10], [15,15], [19,19]], 2)

In [5]:
plot_combined_prob_heat_map_CZT_cart(2, 12, 10)

In [6]:
plot_combined_prob_heat_map_CZT_cart(180, 12, 10)

In [7]:
plot_combined_prob_CZT_cart(2,0,10)

In [8]:
plot_combined_prob_CZT_cart(180,12, 10)

In [9]:
CZT.conv_2(15, 20)

335

In [10]:
len(CZT.pix_x)

16

In [11]:
len(CZT.pix_y)

20

In [12]:
32*40

1280

In [13]:
plot_hist_NaI_cart(180, 0, 10)

In [14]:
plot_spec_NaI_cart(180, 0, 10)

In [15]:
plot_spec_NaI_cart(180, 15, 10)

In [16]:
plot_hist_NaI_cart(180, 15, 10)


In [17]:
NaI.conv_2(10,10)

210

In [18]:
plot_hist_NaI_cart

<function __main__.plot_hist_NaI_cart(e, x, y)>